# LAB 3

We will now use the tools we have learned to perform a simple study of a certain first order ODE problem. The problem comes from Spruce Budworm that has a tendency to periodically eat up fir tree forests in eastern Canada. The original research comes from Ludwig 1976 available here: https://www.jstor.org/stable/3939?seq=1 but is also discussed in a very good book by Strogatz "Nonlinear Dynamics and Chaos". 

## Tasks to be performed:
1. Examine the problem. Try to determine fixed points of the solution.
2. Solve the ODE for different initial conditions. Check what the solution approaches to.
3. Prepare the solution surface as a function of system parameters and determine "refuge", "bistable" and "outbreak" states. Try out different initial conditions.
4. Examine evolution of the system as parameters are slowly varied in time.
5. What should we do with the bugs? Is there a way to prevent the catastrophic situation where the spruce population grows in an uncontrolled manner?

What we intend to learn:
1. Solve simple, first order systems of ODE with Scipy
2. Make plots with MatplotLib
3. Perform root finding for nonlinear problems
4. Other things?

In [ ]:
#Imports we might need
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # Produces 3D axes setting
from matplotlib import cm

from scipy.integrate import odeint
from scipy.optimize import root_scalar

# Introduction
The problem is a variant of a *logistical equation*, or the predator-pray problem in which the population of A is consumed by B according to a simple (but nonlinear) model. We assume population of the worm changes according to:

$$
\dot{N}=RN(1-\frac{N}{K}) - p(N)
$$

where $R$ is the growth parameter, $K$ is the carrying capacity of the ecosystem (how much $N$ can be sustained), and $p(N)$ is the decrease of the population $N$ due to predation by birds. Disregarding $p(N)$ we have a standard, linear logistic equation:

$$
\dot{N}=RN(1-\frac{N}{K})
$$

Following Ludwig 1976 we will adopt a nonlinear predation model:

$$
p(N)=\frac{BN^2}{A^2+N^2}
$$

with $A$ and $B$ some constants. The meaning of this model is such, that for a small worm population birds look for food elsewhere, but when it exceeds a certain value birds simply do not eat the worm fast enough. Or in other words predation saturates.

The entire problem can be rewritten in a non dimensional form:

$$
\dot{x} = rx(1-\frac{x}{k})-\frac{x^2}{1+x^2}
$$

Our task today is to examine this model and to construct the parametric surface $x(r,k)$ as well as to produce some understanding as to how the model behaves. 

Eventually, we will introduce a small change to the model, by allowing $r$ to increase with time at a slow peace, which should correspond to the increase in the foliage cover of the forest (as time passes, there is more trees and more leaves). We should be able to reach the outbreak region, which would result in a "catastrophe". This would be understood as a rapid change in the population of the worm, with a very mild, and continuous change of system parameters. Note: Do you see a relation to the virus outbreak?

**Strogatz, Nonlinear Dynamics and Chaos** section 3.7, p. 79-80:  
Now we need to decide on biologically plausible values of the dimensionless
groups $r=RA/B$ and $k=K/A$. A complication is that these parameters may drift
slowly as the condition of the forest changes. According to Ludwig et al. (1978),
$r$ increases as the forest grows, while $k$ remains fixed.
They reason as follows: (...) the carrying capacity $K$ should
be proportional to the available foliage. (...) parameter $A$ in the predation term should be proportional to $S$; predators such as birds search units of foliage, not acres of forest (...).
The experimental observations suggest that for a young forest, (...) the parameters lie in the bistable region.
The budworm population is kept down by the birds, which find it easy to search the small number of branches per acre. However, as the forest grows (...) $(k, r)$ drifts
upward in parameter space toward the outbreak region (..).
Ludwig et al. (1978) estimate that (..) a fully mature forest (parameters) lies dangerously in the
outbreak region.

# Task 1
Choose $r=0.4$ and $k=14$ as starting values and plot the two parts that constitute the RHS of the model. I.e.:
$
f(x) = rx(1-\frac{x}{k}) \text{ and } p(x)=\frac{x^2}{1+x^2}
$
Determine intersection points of the two curves using `scipy.optimize.root_scalar` and mark them on the plot using `scatter`.

# Task 2
Now examine the sign of the left hand of the ODE and decide which of those three points can pose as a stable solution. All comes down to estimating the sign and deciding should $x$ increase or decrease?

Now let move on, solve the problem and examine if points we determined are attainable depending on the initial condition? We will use the ODE solver for this.

# Task 3
Note the two points "on the outsides" seem like stable solutions. That is solution approaches those in time. How about the middle point? Is it stable? No, it is not, any small deviation will send the system to either of the stable points. Try to integrate using the middle point as initial condition with a very small deviation.

# Task 4
We can now attempt to plot the solution surfaces. We could solve the ODE for each configuration, or just look for roots. We should choose the former solution. It requires us to solve an ODE, but we have no trouble selecting the correct root.

In [ ]:
r = np.linspace(0.01, 1.0, 50)
k = np.linspace(0.01, 20, 50)
R,K = np.meshgrid(r,k)

t = np.linspace(0, 200, 1000)  # one thousand time steps
sh = R.shape
X1 = np.zeros_like(R)
X2 = np.zeros_like(R)
print(R.shape)

Before we start, try to illustrate how the number of roots changes with system parameters. You can do this, by plotting the predation function and than the logistic part for different $k$ and $r$ values.

What we need now is to solve an ODE for each and every $(r,k)$ pair with initial condition chosen such that we obtain both of the stable solutions. You could use the code snippet below:
``` 
x0 = 2
xt1 = odeint(lor_rhs, x0, t)
X1[i,j] = xt1[-1]

x0 = 10
xt2 = odeint(lor_rhs, x0, t)
X2[i,j] = xt2[-1]
```

Plot the 3D surface:

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D # Produces 3D axes setting
from matplotlib import cm

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

surf = ax.plot_surface(R, K, X2, color='red', alpha=1)
surf = ax.plot_surface(R, K, X1, color='blue', alpha=0.5)

plt.show()

# Task 5
Now time for a catastrophe to happen. We will evolve the spruce population in time and assume that conditions of the habitat change slowly in time. That is, as the time goes by, forest grows, there is more leaves to feed on and to hide in. This will translate to a relatively slow change of the habitat caring capacity $k$. We will fix $r=0.4$ and vary $k$ from $5$ up.

In [ ]:
x0 = 0.5
r = 0.6
k = 5
t = np.linspace(0, 200, 1000)

Try to show how the population explodes as parameters $(r,k)$ approach the critical region, we have seen upon finishing Task 4.